In [71]:
excel_file_path = 'z:/Q- Figures&Reports/T32-Poster/Nate/T7-Summary(LocalOld).xlsx'

In [80]:
import pandas as pd
excel_file_path = 'z:/Q- Figures&Reports/T32-Poster/Nate/T7-Summary(LocalOld).xlsx'
# Replace 'your_file.xlsx' with the actual path to your Excel file
# Read the Excel file
data = pd.read_excel(excel_file_path, skiprows=[0])

# Define the indices for the columns you're interested in (0-based indexing)
column2_index = 1  # Replace with the correct index for column 2
column3_index = 2  # Replace with the correct index for column 3

# Filter out rows where column 3 has a value but column 2 is blank
filtered_data = data[data.iloc[:, column2_index].notna() | data.iloc[:, column3_index].isna()]

# Extract the second and third columns from the filtered data and convert them to lists
second_column = filtered_data.iloc[:, column2_index].tolist()
third_column = filtered_data.iloc[:, column3_index].tolist()

# Remove 'NaN' values from the lists
indices = [int(value) for value in second_column if not pd.isna(value)]
case_numbers = [int(value) for value in third_column if not pd.isna(value)]

print("Index:", indices)
print("Case:", case_numbers)


Index: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68]
Case: [22, 24, 28, 29, 30, 32, 34, 36, 38, 39, 42, 43, 44, 46, 31, 48, 49, 50, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 73, 74, 76, 77, 78, 79, 80, 81, 82, 72, 1, 2, 3, 4, 6, 7, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 23, 27, 37]


In [ ]:

def find_files(folder_path):
    vx3_file = None
    mask_file = None
    label_file = None
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file == "Vx3.nrrd":
                vx3_file = os.path.join(root, file)
            elif file.endswith(".mask.nii.gz"):
                mask_file = os.path.join(root, file)
            elif file.endswith("label.nrrd"):
                label_file = os.path.join(root, file)
        
        if vx3_file and mask_file and label_file:
            break  # All files are found, exit the loop
    
    return vx3_file, mask_file, label_file

folder_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-22-0.5_A_80KV_Qr40d_3_53062428/"  # Replace with the actual path to your folder
vx3_file, mask_file, label_file = find_files(folder_path)

if vx3_file and mask_file and label_file:
    print("All files were found.")
    print("Vx3 File:", vx3_file)
    print("Mask File:", mask_file)
    print("Label File:", label_file)
else:
    print("One or more files were not found.")


In [ ]:
# 100 YES WORKING TO FIND ALL THREE FILES CONTAINED WITHIN THE subdirectory 1-DONE
import os
import pandas as pd

excel_file_path = 'z:/Q- Figures&Reports/T32-Poster/Nate/T7-Summary(LocalOld).xlsx'
# Read the Excel file
data = pd.read_excel(excel_file_path, skiprows=[0])

# Define the indices for the columns you're interested in (0-based indexing)
column2_index = 1  # Replace with the correct index for column 2
column3_index = 2  # Replace with the correct index for column 3

# Filter out rows where column 3 has a value but column 2 is blank
filtered_data = data[data.iloc[:, column2_index].notna() | data.iloc[:, column3_index].isna()]

# Extract the second and third columns from the filtered data and convert them to lists
second_column = filtered_data.iloc[:, column2_index].tolist()
third_column = filtered_data.iloc[:, column3_index].tolist()

# Remove 'NaN' values from the lists
indices = [int(value) for value in second_column if not pd.isna(value)]
case_numbers = [int(value) for value in third_column if not pd.isna(value)]

def find_files(folder_path):
    vx3_file = None
    mask_file = None
    label_file = None
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file == "Vx3.nrrd":
                vx3_file = os.path.join(root, file)
            elif file.endswith(".mask.nii.gz"):
                mask_file = os.path.join(root, file)
            elif file.endswith("label.nrrd"):
                label_file = os.path.join(root, file)
        
        if vx3_file and mask_file and label_file:
            break  # All files are found, exit the loop
    
    return vx3_file, mask_file, label_file

base_folder = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/"

for case_number in case_numbers:
    case_folder_prefix = f"Case-{case_number}-"
    
    for folder in os.listdir(base_folder):
        if folder.startswith(case_folder_prefix):
            folder_path = os.path.join(base_folder, folder)
            vx3_file, mask_file, label_file = find_files(folder_path)

            if vx3_file and mask_file and label_file:
                print("All files were found for", folder)
                print("Vx3 File:", vx3_file)
                print("Mask File:", mask_file)
                print("Label File:", label_file)
                print()
            else:
                print("One or more files were not found for", folder)


In [ ]:
import os

# List of case numbers you want to search for
  # Add more case numbers as needed

# Root directory to start the search
root_directory = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/"

# Loop through the case numbers
for case_number in case_numbers:
    case_prefix = f"Case-{case_number}-"
    
    # Function to filter folders that start with the case prefix
    def case_folder_filter(folder_name):
        return folder_name.startswith(case_prefix)

    # Find matching case folders
    matching_case_folders = list(filter(case_folder_filter, os.listdir(root_directory)))

    if not matching_case_folders:
        print(f"No matching folders found for Case {case_number}.")
        continue

    case_folder = os.path.join(root_directory, matching_case_folders[0])

    # Files to search for
    required_files = [".mask.nii.gz", "Vx3.nrrd", "label.nrrd"]

    found_files = []

    # Loop through subfolders of the case folder
    for subdir, _, files in os.walk(case_folder):
        for file in files:
            if any(file.endswith(ext) for ext in required_files):
                found_files.append(file)

    if len(found_files) == len(required_files):
        print(f"All required files found in Case {case_number}.")
    else:
        missing_files = set(required_files) - set(found_files)
        print(f"Missing files in Case {case_number}: {', '.join(missing_files)}")


In [ ]:
# 200 FINAL WORKING CODE FOR IMAGE GENERATION FOR ONE CASE
#############################################################################################################################################################
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import nrrd
from PIL import Image, ImageDraw
import os
import vtk
from mayavi import mlab

# QC IMAGE STUFF
# Casename = "H-14479_JH110088_SPI-JH110088-PBV_SPIROMICS_EXPIRATION_PBV__0.5__QR40__RESEARCH__A_80KV_Qr40d_3_53062428"
Vx3Path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/Vx3.nrrd"
labelpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/S-Final-1-label.nrrd"
maskpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/seg_net/SPI-MU163450-PBV_FRC_0.625_MD STND_VNC.mask.nii.gz"
output_path = 'z:/Q- Figures&Reports/T32-Poster/Nate/Case-59/'
filename = "Case-59-QC_Image.png"
final_filename = "Case-59-QC-Image(2CM).png"
Vx3_path = Vx3Path

# 3D Screenshot Stuff check for repeats
nrrd_file_path = labelpath
segmentation_path = labelpath
case_number = "Case-59-"
output_path_seg = 'z:/Q- Figures&Reports/T32-Poster/Nate/Case-59/'  # Change this to your desired output directory
output_nrrd_file_path = "z:/Q- Figures&Reports/T32-Poster/Nate/Case-59/Aseed.nrrd"

# These filenames are just to have the program run all at the same time
Aseed_filename_base = case_number + "Aseed"
Vseed_filename_base = case_number + "Vseed"

vtk_filename_aseed = f"{Aseed_filename_base}.vtk"
vtk_filename_vseed = f"{Vseed_filename_base}.vtk"
output_filename_aseed = output_path_seg + vtk_filename_aseed
output_filename_vseed = output_path_seg + vtk_filename_vseed
screenshot_filename = "Model.png"


models_folder = os.path.join(os.path.dirname(output_path), "QC-Images")
os.makedirs(models_folder, exist_ok=True)

# Load the image using nibabel to see image information
data, header = nrrd.read(Vx3_path)

# Print the metadata
print("Data Shape:", data.shape)
print("Data Type:", data.dtype)
print("Header:", header)

# Load the 3D image
Vx3 = sitk.ReadImage(Vx3_path)
print("Reading Vx3...")
# Get the size of the image in x, y, and z dimensions
size_x, size_y, size_z = Vx3.GetSize()
spacing = Vx3.GetSpacing()
y_index = Vx3.GetSize()[1] // 2
print("Spacing:", spacing)

# Extract the middle plane along the y-dimension
slice_2d = Vx3[:,y_index,:]
slice_array = sitk.GetArrayFromImage(slice_2d)

# Set the Window level
window_level_min = -1024  # Adjust these values according to your preference
window_level_max = 650

# Apply Window Level to image
windowed_image = sitk.IntensityWindowing(slice_2d, window_level_min, window_level_max, 0, 255)
windowed_slice_array = sitk.GetArrayFromImage(windowed_image)

#Show Vx3
plt.imshow(slice_array, cmap="gray",origin='lower')
plt.axis("off")
plt.show()
print("Window Level Adjusted!")

#Show Vx3 with correct window level
plt.imshow(windowed_slice_array, cmap="gray",origin='lower')
plt.axis("off")
plt.show()

# Save the Corrected Vx3 .PNG to output path
output_file_path = os.path.join(models_folder, filename)
plt.imsave(output_file_path, windowed_slice_array, cmap="gray", origin='lower')
print("Image saved at:", output_file_path)

# Load the image
image_path = output_file_path
output_path_png = models_folder + "/" + final_filename
image = Image.open(image_path)

# Determine line position
line_start_x = 65 # 65 pixels from the left
line_start_y = image.height - 35 # 35 pixels from the bottom

# Calculate line end position
line_end_x = line_start_x + int(20 / 0.38)  # 10 cm converted to pixels
line_end_y = line_start_y

# Draw the white line
draw = ImageDraw.Draw(image)
draw.line([(line_start_x, line_start_y), (line_end_x, line_end_y)], fill='white', width=2)
# # Save the modified image
image.save(output_path_png)

print("Scale bar added and image saved.")

#############################################################################################################################################################


def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)


# This line creates Models Forlder if not already made
if not os.path.exists(output_path_seg):
    os.makedirs(output_path_seg)


# Load the .nrrd segmentation for Vseed
nrrd_reader_vseed = vtk.vtkNrrdReader()
nrrd_reader_vseed.SetFileName(segmentation_path)
nrrd_reader_vseed.Update()

vtk_image_data_vseed = vtk.vtkImageData()
vtk_image_data_vseed.DeepCopy(nrrd_reader_vseed.GetOutput())

# Define scalar value for "Vseed" segment
vseed_scalar_value = 2

# Create a Marching Cubes filter for "Vseed" segment
contour_vseed = vtk.vtkMarchingCubes()
contour_vseed.SetInputData(vtk_image_data_vseed)
contour_vseed.SetValue(0, vseed_scalar_value)
contour_vseed.Update()

# Write the "Vseed" mesh to a .vtk file
vtk_writer_vseed = vtk.vtkPolyDataWriter()
vtk_writer_vseed.SetFileName(output_filename_vseed) 
vtk_writer_vseed.SetInputData(contour_vseed.GetOutput())
vtk_writer_vseed.Write()

print("Vseed Segmentation conversion complete.")

# Read the .nrrd file
data, header = nrrd.read(nrrd_file_path)

data[data == 2] = 0 

# Write the modified data and header to a new .nrrd file
nrrd.write(output_nrrd_file_path, data, header)
print("Aseed.nrrd created!")
# Load the .nrrd segmentation for Aseed
nrrd_reader_aseed = vtk.vtkNrrdReader()
nrrd_reader_aseed.SetFileName(output_nrrd_file_path)
nrrd_reader_aseed.Update()

vtk_image_data_aseed = vtk.vtkImageData()
vtk_image_data_aseed.DeepCopy(nrrd_reader_aseed.GetOutput())

# Define scalar value for "Aseed" segment
aseed_scalar_value = 1

# Create a Marching Cubes filter for "Aseed" segment
contour_aseed = vtk.vtkMarchingCubes()
contour_aseed.SetInputData(vtk_image_data_aseed)
contour_aseed.SetValue(0, aseed_scalar_value)
contour_aseed.Update()

# Write the "Aseed" mesh to a .vtk file
vtk_writer_aseed = vtk.vtkPolyDataWriter()
vtk_writer_aseed.SetFileName(output_filename_aseed) 
vtk_writer_aseed.SetInputData(contour_aseed.GetOutput())
vtk_writer_aseed.Write()

print("Aseed Segmentation Conversion Complete")

#CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
screenshot_output = output_path_seg + case_number + screenshot_filename

mlab.figure(bgcolor=(1, 1, 1))
srcA = mlab.pipeline.open(output_filename_aseed)
srcV = mlab.pipeline.open(output_filename_vseed)

# Create a surface visualization of the data with blue color and 50% opacity
surfaceA = mlab.pipeline.surface(srcA, color=(0, 151/255, 206/255), opacity=1)
surfaceV = mlab.pipeline.surface(srcV, color=(216/255, 101/255, 79/255), opacity=1)

# Find Segmentation Center
model = sitk.ReadImage(segmentation_path)
origin = model.GetOrigin()
dimensions = model.GetSize()
direction = model.GetDirection()
spacing = (0.38, 0.38, 0.38)
distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2


x, y, z = center[0], center[1], center[2]
ypt = y + 500

camera_focal_point = np.array([x, y, z])
camera_position = np.array([x,ypt,z])

# Set the camera parameters
# Write code to calculate this
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]

# Attach the callback to the interaction event
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

magnification_factor = 5
mlab.savefig(screenshot_output, magnification=magnification_factor)


print("Process Completed")

###################################################################################################################################################################

# 2 to fix (nearly done)
## THIS CODE WORKS TO MAKE A MODEL AND POSITION IT AT A FAVORABLE ANGLE TO TAKE AN IMAGE OF THE MAYAVI REPRESENTATION OF THE MODEL

# QC IMAGE STUFF
# Casename = "H-14479_JH110088_SPI-JH110088-PBV_SPIROMICS_EXPIRATION_PBV__0.5__QR40__RESEARCH__A_80KV_Qr40d_3_53062428"
labelpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/S-Final-1-label.nrrd"
filename = "Case-59-QC_Image.png"
final_filename = "Case-59-QC-Image(2CM).png"


# 3D Screenshot Stuff check for repeats
nrrd_file_path = labelpath
segmentation_path = labelpath
case_number = "Case-59-"
output_path_seg = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images/'  # Change this to your desired output directory
# output_nrrd_file_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/Aseed.nrrd"

# Path to the input NIfTI file
input_nifti_path_mask = maskpath
# Path to save the output NRRD file
output_nrrd_path_mask = 'z:/Q- Figures&Reports/T32-Poster/Nate/Case-59/Case-59-Lung-Mask.nrrd'
# Load the NIfTI image
nifti_image = sitk.ReadImage(input_nifti_path_mask)
# Create a writer for NRRD format
nrrd_writer = sitk.ImageFileWriter()
nrrd_writer.SetFileName(output_nrrd_path_mask)
# Convert the NIfTI image to a Numpy array
nifti_array = sitk.GetArrayFromImage(nifti_image)
# Convert the Numpy array back to a SimpleITK image
nrrd_image = sitk.GetImageFromArray(nifti_array)
# Copy information (metadata) from NIfTI image to NRRD image
nrrd_image.CopyInformation(nifti_image)
# Write the NRRD image
nrrd_writer.Execute(nrrd_image)
print("Conversion complete.")

masknrrdpath = output_nrrd_path_mask
output_path = 'z:/Q- Figures&Reports/T32-Poster/Nate/Case-59/'

# 3D Screenshot Stuff check for repeats
segmentation_path = labelpath
case_number = "Case-59-"
output_path_seg = 'z:/Q- Figures&Reports/T32-Poster/Nate/Case-59/'  # Change this to your desired output directory

# These filenames are just to have the program run all at the same time
rLung_filename_base = case_number + "Lungs"
Aseed_filename_base = case_number + "Aseed"

vtk_filename_rLung = f"{rLung_filename_base}.vtk"
vtk_filename_Aseed = f"{Aseed_filename_base}.vtk"
output_filename_rLung = output_path_seg + vtk_filename_rLung
output_filename_Aseed = output_path_seg + vtk_filename_Aseed
lung_screenshot_filename = "Lung-Mask-Model.png"
print("yay")
# This line creates Models Forlder if not already made
if not os.path.exists(output_path_seg):
    os.makedirs(output_path_seg)

data, header = nrrd.read(output_nrrd_path_mask)
data[data == 17] = 3
data[data == 5] = 3
print("yay")
# Load the .nrrd segmentation for Vseed
nrrd_reader_rLung = vtk.vtkNrrdReader()
nrrd_reader_rLung.SetFileName(output_nrrd_path_mask)
nrrd_reader_rLung.Update()

vtk_image_data_rLung = vtk.vtkImageData()
vtk_image_data_rLung.DeepCopy(nrrd_reader_rLung.GetOutput())

# Define scalar value for "Vseed" segment
rLung_scalar_value = 3

# Create a Marching Cubes filter for "Vseed" segment
contour_rLung = vtk.vtkMarchingCubes()
contour_rLung.SetInputData(vtk_image_data_rLung)
contour_rLung.SetValue(0, rLung_scalar_value)
contour_rLung.Update()
print("yay")
# Write the "Vseed" mesh to a .vtk file
vtk_writer_rLung = vtk.vtkPolyDataWriter()
vtk_writer_rLung.SetFileName(output_filename_rLung)
vtk_writer_rLung.SetInputData(contour_rLung.GetOutput())
vtk_writer_rLung.Write()
print("Lung conversion complete.")

#CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
screenshot_output = output_path_seg + case_number + lung_screenshot_filename
mlab.figure(bgcolor=(1, 1, 1))
srcrLung = mlab.pipeline.open(output_filename_rLung)
srcAseed = mlab.pipeline.open(output_filename_Aseed)

# Create a surface visualization of the data with blue color and 50% opacity
surfaceR = mlab.pipeline.surface(srcrLung, color=(128/255, 174/255, 128/255), opacity=.15)
surfaceAseed = mlab.pipeline.surface(srcAseed, color=(0, 151/255, 206/255), opacity=1)

x, y, z = center[0], center[1], center[2]
ypt = y + 500

camera_focal_point = np.array([x, y, z])
camera_position = np.array([x,ypt,z])

print("Camera Position", camera_position)
print("Camera Focal Point", camera_focal_point)

# Set the camera parameters
# Write code to calculate this
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]

print("Origin: ", origin)
print("Dimensions: ", dimensions)
print("Center: ", center)

# Attach the callback to the interaction event

magnification_factor = 5
mlab.savefig(screenshot_output, magnification=magnification_factor)
 
mlab.show()

print("Process Completed")

 

In [ ]:
### WORKING DONT TOUCH. CREATE vseed if that equals 2

import vtk
import os
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import nrrd
from PIL import Image, ImageDraw
from mayavi import mlab


def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)

labelpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/S-Final-1-label.nrrd"
nrrd_file_path = labelpath
filename = "Case-28-QC_Image.png"
final_filename = "Case-28-QC-Image(2CM).png"
segmentation_path = labelpath
case_number = "Case-28-JH112953-SPI__0.75__QR40RESEARCH__A_80KV_Qr40d_53901001"
output_path_seg = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images/'  # Change this to your desired output directory


# This line creates Models Forlder if not already made
if not os.path.exists(output_path_seg):
    os.makedirs(output_path_seg)

# These filenames are just to have the program run all at the same time
Aseed_filename_base = "Aseed"
Vseed_filename_base = "Vseed"
Aseed_model_base = "Aseed"
Vseed_model_base = "Vseed"
vtk_filename_aseed = f"{Aseed_filename_base}.vtk"
vtk_filename_vseed = f"{Vseed_filename_base}.vtk"
output_filename_aseed = output_path_seg + vtk_filename_aseed
output_filename_vseed = output_path_seg + vtk_filename_vseed
screenshot_filename = "-Model.png"


# Load the .nrrd segmentation for Vseed
nrrd_reader_vseed = vtk.vtkNrrdReader()
nrrd_reader_vseed.SetFileName(segmentation_path)
nrrd_reader_vseed.Update()

vtk_image_data_vseed = vtk.vtkImageData()
vtk_image_data_vseed.DeepCopy(nrrd_reader_vseed.GetOutput())

# Define scalar value for "Vseed" segment
vseed_scalar_value = 2

# Create a Marching Cubes filter for "Vseed" segment
contour_vseed = vtk.vtkMarchingCubes()
contour_vseed.SetInputData(vtk_image_data_vseed)
contour_vseed.SetValue(0, vseed_scalar_value)
contour_vseed.Update()

# Write the "Vseed" mesh to a .vtk file
vtk_writer_vseed = vtk.vtkPolyDataWriter()
vtk_writer_vseed.SetFileName(output_filename_vseed) 
vtk_writer_vseed.SetInputData(contour_vseed.GetOutput())
vtk_writer_vseed.Write()

print("Vseed Segmentation conversion complete.")

# Read the .nrrd file
data, header = nrrd.read(nrrd_file_path)

data[data == 2] = 0 

output_nrrd_file_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/Aseed.nrrd"

# Write the modified data and header to a new .nrrd file
nrrd.write(output_nrrd_file_path, data, header)
print("Aseed.nrrd created!")
# Load the .nrrd segmentation for Aseed
nrrd_reader_aseed = vtk.vtkNrrdReader()
nrrd_reader_aseed.SetFileName(output_nrrd_file_path)
nrrd_reader_aseed.Update()

vtk_image_data_aseed = vtk.vtkImageData()
vtk_image_data_aseed.DeepCopy(nrrd_reader_aseed.GetOutput())

# Define scalar value for "Aseed" segment
aseed_scalar_value = 1

# Create a Marching Cubes filter for "Aseed" segment
contour_aseed = vtk.vtkMarchingCubes()
contour_aseed.SetInputData(vtk_image_data_aseed)
contour_aseed.SetValue(0, aseed_scalar_value)
contour_aseed.Update()

# Write the "Aseed" mesh to a .vtk file
vtk_writer_aseed = vtk.vtkPolyDataWriter()
vtk_writer_aseed.SetFileName(output_filename_aseed) 
vtk_writer_aseed.SetInputData(contour_aseed.GetOutput())
vtk_writer_aseed.Write()

print("Aseed Segmentation Conversion Complete")

#CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
screenshot_output = output_path_seg + case_number + screenshot_filename

mlab.figure(bgcolor=(1, 1, 1))
srcA = mlab.pipeline.open(output_filename_aseed)
srcV = mlab.pipeline.open(output_filename_vseed)

# Create a surface visualization of the data with blue color and 50% opacity
surfaceA = mlab.pipeline.surface(srcA, color=(0, 151/255, 206/255), opacity=1)
surfaceV = mlab.pipeline.surface(srcV, color=(216/255, 101/255, 79/255), opacity=1)

# Find Segmentation Center
model = sitk.ReadImage(segmentation_path)
origin = model.GetOrigin()
dimensions = model.GetSize()
direction = model.GetDirection()
spacing = (0.38, 0.38, 0.38)
distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2

print("Origin: ", origin)
print("Dimensions: ", dimensions)
print("Center: ", center)

x, y, z = center[0], center[1], center[2]
ypt = y + 500

camera_focal_point = np.array([x, y, z])
camera_position = np.array([x,ypt,z])
print("Camera Position", camera_position)
print("Camera Focal Point", camera_focal_point)

# Set the camera parameters
# Write code to calculate this
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]

# Attach the callback to the interaction event
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

magnification_factor = 5
mlab.savefig(screenshot_output, magnification=magnification_factor)

# Comment this line below out if you don't want to open 3D viewer
mlab.show()

print("Process Completed")


In [ ]:

import vtk
import os
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import nrrd
from PIL import Image, ImageDraw
from mayavi import mlab


labelpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/S-Final-1-label.nrrd"
maskpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-28-0.75__QR40RESEARCH__A_80KV_Qr40d_53901001/seg_net/SPI-JH112953-PBV_FRC_0.75_Qr40d_VNC.mask.nii.gz"

Vx3_path = Vx3Path
# Output path for PNG of Vx3 
# Replace with the desired output file path
output_path = "z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/"
# filename = "Case-59-QC_Image.png"
# final_filename = "Case-59-QC-Image(2CM).png"

segmentation_path = labelpath
# case_number = "Case-59-JH112953-SPI__0.75__QR40RESEARCH__A_80KV_Qr40d_53901001"
output_path_seg = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images'  # Change this to your desired output directory

# This line creates Models Forlder if not already made
if not os.path.exists(output_path_seg):
    os.makedirs(output_path_seg)

# These filenames are just to have the program run all at the same time
Aseed_filename_base = "Aseed"
Vseed_filename_base = "Vseed"
Aseed_model_base = "Aseed(decimated)"
Vseed_model_base = "Vseed(decimated)"
vtk_filename_aseed = f"{Aseed_filename_base}.vtk"
vtk_filename_vseed = f"{Vseed_filename_base}.vtk"
output_filename_aseed = output_path + vtk_filename_aseed
output_filename_vseed = output_path + vtk_filename_vseed


# Load the .nrrd segmentation for Aseed
nrrd_reader_aseed = vtk.vtkNrrdReader()
nrrd_reader_aseed.SetFileName(segmentation_path)
nrrd_reader_aseed.Update()

vtk_image_data_aseed = vtk.vtkImageData()
vtk_image_data_aseed.DeepCopy(nrrd_reader_aseed.GetOutput())

# Define scalar value for "Aseed" segment
aseed_scalar_value = 1

# Create a Marching Cubes filter for "Aseed" segment
contour_aseed = vtk.vtkMarchingCubes()
contour_aseed.SetInputData(vtk_image_data_aseed)
contour_aseed.SetValue(0, aseed_scalar_value)
contour_aseed.Update()

# Write the "Aseed" mesh to a .vtk file
vtk_writer_aseed = vtk.vtkPolyDataWriter()
vtk_writer_aseed.SetFileName(output_filename_aseed) 
vtk_writer_aseed.SetInputData(contour_aseed.GetOutput())
vtk_writer_aseed.Write()

print("Aseed Segmentation conversion complete.")

In [ ]:

import vtk
import os
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import nrrd
from PIL import Image, ImageDraw
from mayavi import mlab

Vx3Path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-28-0.75__QR40RESEARCH__A_80KV_Qr40d_53901001/Vx3.nrrd"
labelpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-28-0.75__QR40RESEARCH__A_80KV_Qr40d_53901001/Markup/88-Jensen/S-Final-2-label.nrrd"
maskpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-28-0.75__QR40RESEARCH__A_80KV_Qr40d_53901001/seg_net/SPI-JH112953-PBV_FRC_0.75_Qr40d_VNC.mask.nii.gz"

Vx3_path = Vx3Path
# Output path for PNG of Vx3 
# Replace with the desired output file path
output_path = "z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/"
filename = "Case-28-QC_Image.png"
final_filename = "Case-28-QC-Image(2CM).png"

segmentation_path = labelpath
case_number = "Case-28-JH112953-SPI__0.75__QR40RESEARCH__A_80KV_Qr40d_53901001"
output_path_seg = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images'  # Change this to your desired output directory

# This line creates Models Forlder if not already made
if not os.path.exists(output_path_seg):
    os.makedirs(output_path_seg)


# This line creates Models Forlder if not already made
if not os.path.exists(output_path_seg):
    os.makedirs(output_path_seg)

# These filenames are just to have the program run all at the same time
Aseed_filename_base = "Aseed"
Vseed_filename_base = "Vseed"
Aseed_model_base = "Aseed(decimated)"
Vseed_model_base = "Vseed(decimated)"
vtk_filename_aseed = f"{Aseed_filename_base}.vtk"
vtk_filename_vseed = f"{Vseed_filename_base}.vtk"
output_filename_aseed = output_path + vtk_filename_aseed
output_filename_vseed = output_path + vtk_filename_vseed


# Load the .nrrd segmentation for Aseed
nrrd_reader_aseed = vtk.vtkNrrdReader()
nrrd_reader_aseed.SetFileName(segmentation_path)
nrrd_reader_aseed.Update()

vtk_image_data_aseed = vtk.vtkImageData()
vtk_image_data_aseed.DeepCopy(nrrd_reader_aseed.GetOutput())

# Define scalar value for "Aseed" segment
aseed_scalar_value = 2

# Create a Marching Cubes filter for "Aseed" segment
contour_aseed = vtk.vtkMarchingCubes()
contour_aseed.SetInputData(vtk_image_data_aseed)
contour_aseed.SetValue(0, aseed_scalar_value)
contour_aseed.Update()

# Write the "Aseed" mesh to a .vtk file
vtk_writer_aseed = vtk.vtkPolyDataWriter()
vtk_writer_aseed.SetFileName(output_filename_aseed) 
vtk_writer_aseed.SetInputData(contour_aseed.GetOutput())
vtk_writer_aseed.Write()

print("Aseed Segmentation conversion complete.")

In [ ]:
import nrrd
import numpy as np

labelpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-28-0.75__QR40RESEARCH__A_80KV_Qr40d_53901001/Markup/88-Jensen/S-Final-2-label.nrrd"

nrrd_file_path = labelpath

# Read the .nrrd file
data, header = nrrd.read(nrrd_file_path)

data[data == 2] = 0 

output_nrrd_file_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-28-0.75__QR40RESEARCH__A_80KV_Qr40d_53901001/Markup/88-Jensen/Aseed.nrrd"

# Write the modified data and header to a new .nrrd file
nrrd.write(output_nrrd_file_path, data, header)
print("Aseed.nrrd created!")
# Load the .nrrd segmentation for Aseed
nrrd_reader_aseed = vtk.vtkNrrdReader()
nrrd_reader_aseed.SetFileName(output_nrrd_file_path)
nrrd_reader_aseed.Update()

vtk_image_data_aseed = vtk.vtkImageData()
vtk_image_data_aseed.DeepCopy(nrrd_reader_aseed.GetOutput())

# Define scalar value for "Aseed" segment
aseed_scalar_value = 1

# Create a Marching Cubes filter for "Aseed" segment
contour_aseed = vtk.vtkMarchingCubes()
contour_aseed.SetInputData(vtk_image_data_aseed)
contour_aseed.SetValue(0, aseed_scalar_value)
contour_aseed.Update()

# Write the "Aseed" mesh to a .vtk file
vtk_writer_aseed = vtk.vtkPolyDataWriter()
vtk_writer_aseed.SetFileName(output_filename_aseed) 
vtk_writer_aseed.SetInputData(contour_aseed.GetOutput())
vtk_writer_aseed.Write()

In [ ]:
# Make model of Lung Mask from the .nrrd Lung Mask

#############################################################################################################################################################
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import nrrd
from PIL import Image, ImageDraw
import os
import vtk
from mayavi import mlab

# QC IMAGE STUFF
# Casename = "H-14479_JH110088_SPI-JH110088-PBV_SPIROMICS_EXPIRATION_PBV__0.5__QR40__RESEARCH__A_80KV_Qr40d_3_53062428"

Vx3Path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/Vx3.nrrd"
labelpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/S-Final-1-label.nrrd"
maskpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/seg_net/SPI-MU163450-PBV_FRC_0.625_MD STND_VNC.mask.nii.gz"
output_path = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images/'
filename = "Case-59-QC_Image.png"
final_filename = "Case-59-QC-Image(2CM).png"
Vx3_path = Vx3Path

# 3D Screenshot Stuff check for repeats
nrrd_file_path = maskpath
segmentation_path = labelpath
case_number = "Case-59-"
output_path_seg = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images/'  # Change this to your desired output directory
output_nrrd_file_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/(LUNG MASK).nrrd"

 
# These filenames are just to have the program run all at the same time
lLung_filename_base = case_number + "lLung"
rLung_filename_base = case_number + "rLung"


vtk_filename_lLung = f"{lLung_filename_base}.vtk"
vtk_filename_rLung = f"{rLung_filename_base}.vtk"
output_filename_lLung = output_path_seg + vtk_filename_lLung
output_filename_rLung = output_path_seg + vtk_filename_rLung
screenshot_filename = "Model.png"

def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)

# This line creates Models Forlder if not already made
if not os.path.exists(output_path_seg):
    os.makedirs(output_path_seg)

# Load the .nrrd segmentation for Vseed
nrrd_reader_rLung = vtk.vtkNrrdReader()
nrrd_reader_rLung.SetFileName(maskpath)
nrrd_reader_rLung.Update()

vtk_image_data_rLung = vtk.vtkImageData()
vtk_image_data_rLung.DeepCopy(nrrd_reader_rLung.GetOutput())

# Define scalar value for "Vseed" segment
rLung_scalar_value = 17

# Create a Marching Cubes filter for "Vseed" segment
contour_rLung = vtk.vtkMarchingCubes()
contour_rLung.SetInputData(vtk_image_data_rLung)
contour_rLung.SetValue(0, rLung_scalar_value)
contour_rLung.Update()

# Write the "Vseed" mesh to a .vtk file
vtk_writer_rLung = vtk.vtkPolyDataWriter()
vtk_writer_rLung.SetFileName(output_filename_rLung)
vtk_writer_rLung.SetInputData(contour_rLung.GetOutput())
vtk_writer_rLung.Write()

print("Vseed Segmentation conversion complete.")

# Read the .nrrd file
data, header = nrrd.read(nrrd_file_path)
data[data == 17] = 0

# Write the modified data and header to a new .nrrd file
nrrd.write(output_nrrd_file_path, data, header)
print("Aseed.nrrd created!")

# Load the .nrrd segmentation for Aseed
nrrd_reader_lLung = vtk.vtkNrrdReader()
nrrd_reader_lLung.SetFileName(output_nrrd_file_path) ######FIX THIS
nrrd_reader_lLung.Update()

vtk_image_data_lLung = vtk.vtkImageData()
vtk_image_data_lLung.DeepCopy(nrrd_reader_lLung.GetOutput())

# Define scalar value for "Aseed" segment
lLung_scalar_value = 5

# Create a Marching Cubes filter for "Aseed" segment
contour_lLung = vtk.vtkMarchingCubes()
contour_lLung.SetInputData(vtk_image_data_lLung)
contour_lLung.SetValue(0, lLung_scalar_value)
contour_lLung.Update()

# Write the "Aseed" mesh to a .vtk file
vtk_writer_lLung = vtk.vtkPolyDataWriter()
vtk_writer_lLung.SetFileName(output_filename_lLung)
vtk_writer_lLung.SetInputData(contour_lLung.GetOutput())
vtk_writer_lLung.Write()

print("Aseed Segmentation Conversion Complete")

#CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
screenshot_output = output_path_seg + case_number + screenshot_filename
mlab.figure(bgcolor=(1, 1, 1))
srclLung = mlab.pipeline.open(output_filename_lLung)
srcrLung = mlab.pipeline.open(output_filename_rLung)

# Create a surface visualization of the data with blue color and 50% opacity
surfaceL = mlab.pipeline.surface(srclLung, color=(0, 151/255, 206/255), opacity=1)
surfaceR = mlab.pipeline.surface(srcrLung, color=(216/255, 101/255, 79/255), opacity=1)

# Find Segmentation Center
model = sitk.ReadImage(segmentation_path)
origin = model.GetOrigin()
dimensions = model.GetSize()
direction = model.GetDirection()
spacing = (0.38, 0.38, 0.38)
distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2

print("Origin: ", origin)
print("Dimensions: ", dimensions)
print("Center: ", center)

x, y, z = center[0], center[1], center[2]
ypt = y + 500

camera_focal_point = np.array([x, y, z])
camera_position = np.array([x,ypt,z])

print("Camera Position", camera_position)
print("Camera Focal Point", camera_focal_point)

# Set the camera parameters
# Write code to calculate this
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]

# Attach the callback to the interaction event
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)
magnification_factor = 5
mlab.savefig(screenshot_output, magnification=magnification_factor)
 
mlab.show()

print("Process Completed")

In [ ]:
# Make model of Lung Mask from the .nrrd Lung Mask

#############################################################################################################################################################
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import nrrd
from PIL import Image, ImageDraw
import os
import vtk
from mayavi import mlab

# QC IMAGE STUFF
# Casename = "H-14479_JH110088_SPI-JH110088-PBV_SPIROMICS_EXPIRATION_PBV__0.5__QR40__RESEARCH__A_80KV_Qr40d_3_53062428"
labelpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/S-Final-1-label.nrrd"
maskpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/seg_net/SPI-MU163450-PBV_FRC_0.625_MD STND_VNC.mask.nii.gz"
output_path = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images/'
filename = "Case-59-QC_Image.png"
final_filename = "Case-59-QC-Image(2CM).png"
Vx3_path = Vx3Path

# 3D Screenshot Stuff check for repeats
nrrd_file_path = maskpath
segmentation_path = labelpath
case_number = "Case-59-"
output_path_seg = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images/'  # Change this to your desired output directory
output_nrrd_file_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/(LUNG MASK).nrrd"

 
# These filenames are just to have the program run all at the same time
lLung_filename_base = case_number + "lLung"
rLung_filename_base = case_number + "rLung"


vtk_filename_lLung = f"{lLung_filename_base}.vtk"
vtk_filename_rLung = f"{rLung_filename_base}.vtk"
output_filename_lLung = output_path_seg + vtk_filename_lLung
output_filename_rLung = output_path_seg + vtk_filename_rLung
screenshot_filename = "Model.png"

def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)

# This line creates Models Forlder if not already made
if not os.path.exists(output_path_seg):
    os.makedirs(output_path_seg)

# Load the .nrrd segmentation for Vseed
nrrd_reader_rLung = vtk.vtkNrrdReader()
nrrd_reader_rLung.SetFileName(maskpath)
nrrd_reader_rLung.Update()

vtk_image_data_rLung = vtk.vtkImageData()
vtk_image_data_rLung.DeepCopy(nrrd_reader_rLung.GetOutput())

# Define scalar value for "Vseed" segment
rLung_scalar_value = 17

# Create a Marching Cubes filter for "Vseed" segment
contour_rLung = vtk.vtkMarchingCubes()
contour_rLung.SetInputData(vtk_image_data_rLung)
contour_rLung.SetValue(0, rLung_scalar_value)
contour_rLung.Update()

# Write the "Vseed" mesh to a .vtk file
vtk_writer_rLung = vtk.vtkPolyDataWriter()
vtk_writer_rLung.SetFileName(output_filename_rLung)
vtk_writer_rLung.SetInputData(contour_rLung.GetOutput())
vtk_writer_rLung.Write()

print("Vseed Segmentation conversion complete.")

# Read the .nrrd file
data, header = nrrd.read(nrrd_file_path)
data[data == 17] = 0

# Write the modified data and header to a new .nrrd file
nrrd.write(output_nrrd_file_path, data, header)
print("Aseed.nrrd created!")

# Load the .nrrd segmentation for Aseed
nrrd_reader_lLung = vtk.vtkNrrdReader()
nrrd_reader_lLung.SetFileName(output_nrrd_file_path) ######FIX THIS
nrrd_reader_lLung.Update()

vtk_image_data_lLung = vtk.vtkImageData()
vtk_image_data_lLung.DeepCopy(nrrd_reader_lLung.GetOutput())

# Define scalar value for "Aseed" segment
lLung_scalar_value = 5

# Create a Marching Cubes filter for "Aseed" segment
contour_lLung = vtk.vtkMarchingCubes()
contour_lLung.SetInputData(vtk_image_data_lLung)
contour_lLung.SetValue(0, lLung_scalar_value)
contour_lLung.Update()

# Write the "Aseed" mesh to a .vtk file
vtk_writer_lLung = vtk.vtkPolyDataWriter()
vtk_writer_lLung.SetFileName(output_filename_lLung)
vtk_writer_lLung.SetInputData(contour_lLung.GetOutput())
vtk_writer_lLung.Write()

print("Aseed Segmentation Conversion Complete")

#CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
screenshot_output = output_path_seg + case_number + screenshot_filename
mlab.figure(bgcolor=(1, 1, 1))
srclLung = mlab.pipeline.open(output_filename_lLung)
srcrLung = mlab.pipeline.open(output_filename_rLung)

# Create a surface visualization of the data with blue color and 50% opacity
surfaceL = mlab.pipeline.surface(srclLung, color=(0, 151/255, 206/255), opacity=1)
surfaceR = mlab.pipeline.surface(srcrLung, color=(216/255, 101/255, 79/255), opacity=1)

# Find Segmentation Center
model = sitk.ReadImage(segmentation_path)
origin = model.GetOrigin()
dimensions = model.GetSize()
direction = model.GetDirection()
spacing = (0.38, 0.38, 0.38)
distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2

print("Origin: ", origin)
print("Dimensions: ", dimensions)
print("Center: ", center)

x, y, z = center[0], center[1], center[2]
ypt = y + 500

camera_focal_point = np.array([x, y, z])
camera_position = np.array([x,ypt,z])

print("Camera Position", camera_position)
print("Camera Focal Point", camera_focal_point)

# Set the camera parameters
# Write code to calculate this
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]

# Attach the callback to the interaction event
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)
magnification_factor = 5
mlab.savefig(screenshot_output, magnification=magnification_factor)
 
mlab.show()

print("Process Completed")

In [ ]:
# YES THIS WORKS TO CREATE .NRRD FROM .NII.GZ Mask

import SimpleITK as sitk

# Path to the input NIfTI file
input_nifti_path = 'z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/seg_net/SPI-MU163450-PBV_FRC_0.625_MD STND_VNC.mask.nii.gz'

# Path to save the output NRRD file
output_nrrd_path = 'z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/seg_net/outputlung.nrrd'

# Load the NIfTI image
nifti_image = sitk.ReadImage(input_nifti_path)

# Create a writer for NRRD format
nrrd_writer = sitk.ImageFileWriter()
nrrd_writer.SetFileName(output_nrrd_path)

# Convert the NIfTI image to a Numpy array
nifti_array = sitk.GetArrayFromImage(nifti_image)

# Convert the Numpy array back to a SimpleITK image
nrrd_image = sitk.GetImageFromArray(nifti_array)

# Copy information (metadata) from NIfTI image to NRRD image
nrrd_image.CopyInformation(nifti_image)

# Write the NRRD image
nrrd_writer.Execute(nrrd_image)

print("Conversion complete.")


In [ ]:
# YES Make model of Lung Mask from the .nrrd Lung Mask

#############################################################################################################################################################
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import nrrd
from PIL import Image, ImageDraw
import os
import vtk
from mayavi import mlab

# QC IMAGE STUFF
# Casename = "H-14479_JH110088_SPI-JH110088-PBV_SPIROMICS_EXPIRATION_PBV__0.5__QR40__RESEARCH__A_80KV_Qr40d_3_53062428"
labelpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/S-Final-1-label.nrrd"
masknrrdpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/seg_net/outputlung.nrrd"
output_path = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images/'
filename = "Case-59-QC_Image.png"
final_filename = "Case-59-QC-Image(2CM).png"
Vx3_path = Vx3Path

# 3D Screenshot Stuff check for repeats
segmentation_path = labelpath
case_number = "Case-59-"
output_path_seg = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images/'  # Change this to your desired output directory
output_nrrd_file_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/(LeftLung).nrrd"

 
# These filenames are just to have the program run all at the same time
lLung_filename_base = case_number + "lLung"
rLung_filename_base = case_number + "rLung"
Aseed_filename_base = case_number + "Aseed"

vtk_filename_lLung = f"{lLung_filename_base}.vtk"
vtk_filename_rLung = f"{rLung_filename_base}.vtk"
vtk_filename_Aseed = f"{Aseed_filename_base}.vtk"
output_filename_lLung = output_path_seg + vtk_filename_lLung
output_filename_rLung = output_path_seg + vtk_filename_rLung
output_filename_Aseed = output_path_seg + vtk_filename_Aseed
screenshot_filename = "Model.png"

def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)

# This line creates Models Forlder if not already made
if not os.path.exists(output_path_seg):
    os.makedirs(output_path_seg)

# Load the .nrrd segmentation for Vseed
nrrd_reader_rLung = vtk.vtkNrrdReader()
nrrd_reader_rLung.SetFileName(masknrrdpath)
nrrd_reader_rLung.Update()

vtk_image_data_rLung = vtk.vtkImageData()
vtk_image_data_rLung.DeepCopy(nrrd_reader_rLung.GetOutput())

# Define scalar value for "Vseed" segment
rLung_scalar_value = 17

# Create a Marching Cubes filter for "Vseed" segment
contour_rLung = vtk.vtkMarchingCubes()
contour_rLung.SetInputData(vtk_image_data_rLung)
contour_rLung.SetValue(0, rLung_scalar_value)
contour_rLung.Update()

# Write the "Vseed" mesh to a .vtk file
vtk_writer_rLung = vtk.vtkPolyDataWriter()
vtk_writer_rLung.SetFileName(output_filename_rLung)
vtk_writer_rLung.SetInputData(contour_rLung.GetOutput())
vtk_writer_rLung.Write()

print("Vseed Segmentation conversion complete.")

# Read the .nrrd file
data, header = nrrd.read(masknrrdpath)
data[data == 17] = 0

# Write the modified data and header to a new .nrrd file
nrrd.write(output_nrrd_file_path, data, header)
print("Aseed.nrrd created!")

# Load the .nrrd segmentation for Aseed
nrrd_reader_lLung = vtk.vtkNrrdReader()
nrrd_reader_lLung.SetFileName(output_nrrd_file_path) ######FIX THIS
nrrd_reader_lLung.Update()

vtk_image_data_lLung = vtk.vtkImageData()
vtk_image_data_lLung.DeepCopy(nrrd_reader_lLung.GetOutput())

# Define scalar value for "Aseed" segment
lLung_scalar_value = 5

# Create a Marching Cubes filter for "Aseed" segment
contour_lLung = vtk.vtkMarchingCubes()
contour_lLung.SetInputData(vtk_image_data_lLung)
contour_lLung.SetValue(0, lLung_scalar_value)
contour_lLung.Update()

# Write the "Aseed" mesh to a .vtk file
vtk_writer_lLung = vtk.vtkPolyDataWriter()
vtk_writer_lLung.SetFileName(output_filename_lLung)
vtk_writer_lLung.SetInputData(contour_lLung.GetOutput())
vtk_writer_lLung.Write()

print("Aseed Segmentation Conversion Complete")

#CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
screenshot_output = output_path_seg + case_number + screenshot_filename
mlab.figure(bgcolor=(1, 1, 1))
srclLung = mlab.pipeline.open(output_filename_lLung)
srcrLung = mlab.pipeline.open(output_filename_rLung)
srcAseed = mlab.pipeline.open(output_filename_Aseed)

# Create a surface visualization of the data with blue color and 50% opacity
surfaceL = mlab.pipeline.surface(srclLung, color=(199/255, 145/255, 28/255), opacity=.15)
surfaceR = mlab.pipeline.surface(srcrLung, color=(199/255, 145/255, 28/255), opacity=.15)
surfaceAseed = mlab.pipeline.surface(srcAseed, color=(0, 151/255, 206/255), opacity=1)

# Find Segmentation Center
model = sitk.ReadImage(segmentation_path)
origin = model.GetOrigin()
dimensions = model.GetSize()
direction = model.GetDirection()
spacing = (0.38, 0.38, 0.38)
distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2

print("Origin: ", origin)
print("Dimensions: ", dimensions)
print("Center: ", center)

x, y, z = center[0], center[1], center[2]
ypt = y + 500

camera_focal_point = np.array([x, y, z])
camera_position = np.array([x,ypt,z])

print("Camera Position", camera_position)
print("Camera Focal Point", camera_focal_point)

# Set the camera parameters
# Write code to calculate this
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]

# Attach the callback to the interaction event
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)
magnification_factor = 5
mlab.savefig(screenshot_output, magnification=magnification_factor)
 
mlab.show()

print("Process Completed")

In [ ]:
#1 FINAL WORKING CODE FOR IMAGE GENERATION FOR ONE CASE


#############################################################################################################################################################
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import nrrd
from PIL import Image, ImageDraw
import os
import vtk
from mayavi import mlab

# QC IMAGE STUFF
# Casename = "H-14479_JH110088_SPI-JH110088-PBV_SPIROMICS_EXPIRATION_PBV__0.5__QR40__RESEARCH__A_80KV_Qr40d_3_53062428"
Vx3Path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/Vx3.nrrd"
labelpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/S-Final-1-label.nrrd"
maskpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/seg_net/SPI-MU163450-PBV_FRC_0.625_MD STND_VNC.mask.nii.gz"
output_path = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images/'
filename = "Case-59-QC_Image.png"
final_filename = "Case-59-QC-Image(2CM).png"
Vx3_path = Vx3Path

# 3D Screenshot Stuff check for repeats
nrrd_file_path = labelpath
segmentation_path = labelpath
case_number = "Case-59-"
output_path_seg = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images/'  # Change this to your desired output directory
output_nrrd_file_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/Aseed.nrrd"

# These filenames are just to have the program run all at the same time
Aseed_filename_base = case_number + "Aseed"
Vseed_filename_base = case_number + "Vseed"

vtk_filename_aseed = f"{Aseed_filename_base}.vtk"
vtk_filename_vseed = f"{Vseed_filename_base}.vtk"
output_filename_aseed = output_path_seg + vtk_filename_aseed
output_filename_vseed = output_path_seg + vtk_filename_vseed
screenshot_filename = "Model.png"


models_folder = os.path.join(os.path.dirname(output_path), "QC-Images")
os.makedirs(models_folder, exist_ok=True)

# Load the image using nibabel to see image information
data, header = nrrd.read(Vx3_path)

# Print the metadata
print("Data Shape:", data.shape)
print("Data Type:", data.dtype)
print("Header:", header)

# Load the 3D image
Vx3 = sitk.ReadImage(Vx3_path)
print("Reading Vx3...")
# Get the size of the image in x, y, and z dimensions
size_x, size_y, size_z = Vx3.GetSize()
spacing = Vx3.GetSpacing()
y_index = Vx3.GetSize()[1] // 2
print("Spacing:", spacing)

# Extract the middle plane along the y-dimension
slice_2d = Vx3[:,y_index,:]
slice_array = sitk.GetArrayFromImage(slice_2d)

# Set the Window level
window_level_min = -1024  # Adjust these values according to your preference
window_level_max = 650

# Apply Window Level to image
windowed_image = sitk.IntensityWindowing(slice_2d, window_level_min, window_level_max, 0, 255)
windowed_slice_array = sitk.GetArrayFromImage(windowed_image)

#Show Vx3
plt.imshow(slice_array, cmap="gray",origin='lower')
plt.axis("off")
plt.show()
print("Window Level Adjusted!")

#Show Vx3 with correct window level
plt.imshow(windowed_slice_array, cmap="gray",origin='lower')
plt.axis("off")
plt.show()

# Save the Corrected Vx3 .PNG to output path
output_file_path = os.path.join(models_folder, filename)
plt.imsave(output_file_path, windowed_slice_array, cmap="gray", origin='lower')
print("Image saved at:", output_file_path)

# Load the image
image_path = output_file_path
output_path_png = models_folder + "/" + final_filename
image = Image.open(image_path)

# Determine line position
line_start_x = 65 # 65 pixels from the left
line_start_y = image.height - 35 # 35 pixels from the bottom

# Calculate line end position
line_end_x = line_start_x + int(20 / 0.38)  # 10 cm converted to pixels
line_end_y = line_start_y

# Draw the white line
draw = ImageDraw.Draw(image)
draw.line([(line_start_x, line_start_y), (line_end_x, line_end_y)], fill='white', width=2)
# # Save the modified image
image.save(output_path_png)

print("Scale bar added and image saved.")

#############################################################################################################################################################

#CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
screenshot_output = output_path_seg + case_number + screenshot_filename

def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)


# This line creates Models Forlder if not already made
if not os.path.exists(output_path_seg):
    os.makedirs(output_path_seg)


# Load the .nrrd segmentation for Vseed
nrrd_reader_vseed = vtk.vtkNrrdReader()
nrrd_reader_vseed.SetFileName(segmentation_path)
nrrd_reader_vseed.Update()

vtk_image_data_vseed = vtk.vtkImageData()
vtk_image_data_vseed.DeepCopy(nrrd_reader_vseed.GetOutput())

# Define scalar value for "Vseed" segment
vseed_scalar_value = 2

# Create a Marching Cubes filter for "Vseed" segment
contour_vseed = vtk.vtkMarchingCubes()
contour_vseed.SetInputData(vtk_image_data_vseed)
contour_vseed.SetValue(0, vseed_scalar_value)
contour_vseed.Update()

# Write the "Vseed" mesh to a .vtk file
vtk_writer_vseed = vtk.vtkPolyDataWriter()
vtk_writer_vseed.SetFileName(output_filename_vseed) 
vtk_writer_vseed.SetInputData(contour_vseed.GetOutput())
vtk_writer_vseed.Write()

print("Vseed Segmentation conversion complete.")

# Read the .nrrd file
data, header = nrrd.read(nrrd_file_path)

data[data == 2] = 0 

# Write the modified data and header to a new .nrrd file
nrrd.write(output_nrrd_file_path, data, header)
print("Aseed.nrrd created!")
# Load the .nrrd segmentation for Aseed
nrrd_reader_aseed = vtk.vtkNrrdReader()
nrrd_reader_aseed.SetFileName(output_nrrd_file_path)
nrrd_reader_aseed.Update()

vtk_image_data_aseed = vtk.vtkImageData()
vtk_image_data_aseed.DeepCopy(nrrd_reader_aseed.GetOutput())

# Define scalar value for "Aseed" segment
aseed_scalar_value = 1

# Create a Marching Cubes filter for "Aseed" segment
contour_aseed = vtk.vtkMarchingCubes()
contour_aseed.SetInputData(vtk_image_data_aseed)
contour_aseed.SetValue(0, aseed_scalar_value)
contour_aseed.Update()

# Write the "Aseed" mesh to a .vtk file
vtk_writer_aseed = vtk.vtkPolyDataWriter()
vtk_writer_aseed.SetFileName(output_filename_aseed) 
vtk_writer_aseed.SetInputData(contour_aseed.GetOutput())
vtk_writer_aseed.Write()

print("Aseed Segmentation Conversion Complete")


mlab.figure(bgcolor=(1, 1, 1))
srcA = mlab.pipeline.open(output_filename_aseed)
srcV = mlab.pipeline.open(output_filename_vseed)

# Create a surface visualization of the data with blue color and 50% opacity
surfaceA = mlab.pipeline.surface(srcA, color=(0, 151/255, 206/255), opacity=1)
surfaceV = mlab.pipeline.surface(srcV, color=(216/255, 101/255, 79/255), opacity=1)

# Find Segmentation Center
model = sitk.ReadImage(segmentation_path)
origin = model.GetOrigin()
dimensions = model.GetSize()
direction = model.GetDirection()
spacing = (0.38, 0.38, 0.38)
distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2

print("Origin: ", origin)
print("Dimensions: ", dimensions)
print("Center: ", center)

x, y, z = center[0], center[1], center[2]
ypt = y + 500

camera_focal_point = np.array([x, y, z])
camera_position = np.array([x,ypt,z])
print("Camera Position", camera_position)
print("Camera Focal Point", camera_focal_point)

# Set the camera parameters
# Write code to calculate this
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]


magnification_factor = 5
mlab.savefig(screenshot_output, magnification=magnification_factor)


print("Process Completed")

###################################################################################################################################################################

# 2 to fix (nearly done)
## THIS CODE WORKS TO MAKE A MODEL AND POSITION IT AT A FAVORABLE ANGLE TO TAKE AN IMAGE OF THE MAYAVI REPRESENTATION OF THE MODEL

# Path to the input NIfTI file
input_nifti_path_mask = maskpath
# Path to save the output NRRD file
output_nrrd_path_mask = 'z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/seg_net/outputlung.nrrd'
# Load the NIfTI image
nifti_image = sitk.ReadImage(input_nifti_path_mask)
# Create a writer for NRRD format
nrrd_writer = sitk.ImageFileWriter()
nrrd_writer.SetFileName(output_nrrd_path)
# Convert the NIfTI image to a Numpy array
nifti_array = sitk.GetArrayFromImage(nifti_image)
# Convert the Numpy array back to a SimpleITK image
nrrd_image = sitk.GetImageFromArray(nifti_array)
# Copy information (metadata) from NIfTI image to NRRD image
nrrd_image.CopyInformation(nifti_image)
# Write the NRRD image
nrrd_writer.Execute(nrrd_image)
print("Conversion complete.")


labelpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/S-Final-1-label.nrrd"
masknrrdpath = output_nrrd_path_mask
output_path = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images/'

Vx3_path = Vx3Path

# 3D Screenshot Stuff check for repeats
segmentation_path = labelpath
case_number = "Case-59-"
output_path_seg = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images/'  # Change this to your desired output directory
output_nrrd_file_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/(LeftLung).nrrd"

# These filenames are just to have the program run all at the same time
lLung_filename_base = case_number + "lLung"
rLung_filename_base = case_number + "rLung"
Aseed_filename_base = case_number + "Aseed"

vtk_filename_lLung = f"{lLung_filename_base}.vtk"
vtk_filename_rLung = f"{rLung_filename_base}.vtk"
vtk_filename_Aseed = f"{Aseed_filename_base}.vtk"
output_filename_lLung = output_path_seg + vtk_filename_lLung
output_filename_rLung = output_path_seg + vtk_filename_rLung
output_filename_Aseed = output_path_seg + vtk_filename_Aseed
screenshot_filename = "Model.png"

# This line creates Models Forlder if not already made
if not os.path.exists(output_path_seg):
    os.makedirs(output_path_seg)

# Load the .nrrd segmentation for Vseed
nrrd_reader_rLung = vtk.vtkNrrdReader()
nrrd_reader_rLung.SetFileName(masknrrdpath)
nrrd_reader_rLung.Update()

vtk_image_data_rLung = vtk.vtkImageData()
vtk_image_data_rLung.DeepCopy(nrrd_reader_rLung.GetOutput())

# Define scalar value for "Vseed" segment
rLung_scalar_value = 17

# Create a Marching Cubes filter for "Vseed" segment
contour_rLung = vtk.vtkMarchingCubes()
contour_rLung.SetInputData(vtk_image_data_rLung)
contour_rLung.SetValue(0, rLung_scalar_value)
contour_rLung.Update()

# Write the "Vseed" mesh to a .vtk file
vtk_writer_rLung = vtk.vtkPolyDataWriter()
vtk_writer_rLung.SetFileName(output_filename_rLung)
vtk_writer_rLung.SetInputData(contour_rLung.GetOutput())
vtk_writer_rLung.Write()

print("Vseed Segmentation conversion complete.")

# Read the .nrrd file
data, header = nrrd.read(masknrrdpath)
data[data == 17] = 0

# Write the modified data and header to a new .nrrd file
nrrd.write(output_nrrd_file_path, data, header)
print("Aseed.nrrd created!")

# Load the .nrrd segmentation for Aseed
nrrd_reader_lLung = vtk.vtkNrrdReader()
nrrd_reader_lLung.SetFileName(output_nrrd_file_path) ######FIX THIS
nrrd_reader_lLung.Update()

vtk_image_data_lLung = vtk.vtkImageData()
vtk_image_data_lLung.DeepCopy(nrrd_reader_lLung.GetOutput())

# Define scalar value for "Aseed" segment
lLung_scalar_value = 5

# Create a Marching Cubes filter for "Aseed" segment
contour_lLung = vtk.vtkMarchingCubes()
contour_lLung.SetInputData(vtk_image_data_lLung)
contour_lLung.SetValue(0, lLung_scalar_value)
contour_lLung.Update()

# Write the "Aseed" mesh to a .vtk file
vtk_writer_lLung = vtk.vtkPolyDataWriter()
vtk_writer_lLung.SetFileName(output_filename_lLung)
vtk_writer_lLung.SetInputData(contour_lLung.GetOutput())
vtk_writer_lLung.Write()

print("Aseed Segmentation Conversion Complete")

#CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
screenshot_output = output_path_seg + case_number + screenshot_filename
mlab.figure(bgcolor=(1, 1, 1))
srclLung = mlab.pipeline.open(output_filename_lLung)
srcrLung = mlab.pipeline.open(output_filename_rLung)
srcAseed = mlab.pipeline.open(output_filename_Aseed)

# Create a surface visualization of the data with blue color and 50% opacity
surfaceL = mlab.pipeline.surface(srclLung, color=(199/255, 145/255, 28/255), opacity=.15)
surfaceR = mlab.pipeline.surface(srcrLung, color=(199/255, 145/255, 28/255), opacity=.15)
surfaceAseed = mlab.pipeline.surface(srcAseed, color=(0, 151/255, 206/255), opacity=1)

# Find Segmentation Center
model = sitk.ReadImage(segmentation_path)
origin = model.GetOrigin()
dimensions = model.GetSize()
direction = model.GetDirection()
spacing = (0.38, 0.38, 0.38)
distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2

print("Origin: ", origin)
print("Dimensions: ", dimensions)
print("Center: ", center)

x, y, z = center[0], center[1], center[2]
ypt = y + 500

camera_focal_point = np.array([x, y, z])
camera_position = np.array([x,ypt,z])

print("Camera Position", camera_position)
print("Camera Focal Point", camera_focal_point)

# Set the camera parameters
# Write code to calculate this
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]

# Attach the callback to the interaction event
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)
magnification_factor = 5
mlab.savefig(screenshot_output, magnification=magnification_factor)
 
mlab.show()

print("Process Completed")


In [ ]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import nrrd
from PIL import Image, ImageDraw
import os
import vtk
from mayavi import mlab

# QC IMAGE STUFF
# Casename = "H-14479_JH110088_SPI-JH110088-PBV_SPIROMICS_EXPIRATION_PBV__0.5__QR40__RESEARCH__A_80KV_Qr40d_3_53062428"
labelpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/S-Final-1-label.nrrd"
filename = "Case-59-QC_Image.png"
final_filename = "Case-59-QC-Image(2CM).png"


# 3D Screenshot Stuff check for repeats
nrrd_file_path = labelpath
segmentation_path = labelpath
case_number = "Case-59-"
output_path_seg = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images/'  # Change this to your desired output directory
output_nrrd_file_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/Aseed.nrrd"

maskpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/seg_net/SPI-MU163450-PBV_FRC_0.625_MD STND_VNC.mask.nii.gz"
# Path to the input NIfTI file
input_nifti_path_mask = maskpath
# Path to save the output NRRD file
output_nrrd_path_mask = 'z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/seg_net/Case-59-Lung-Mask.nrrd'
# Load the NIfTI image
nifti_image = sitk.ReadImage(input_nifti_path_mask)
# Create a writer for NRRD format
nrrd_writer = sitk.ImageFileWriter()
nrrd_writer.SetFileName(output_nrrd_path_mask)
# Convert the NIfTI image to a Numpy array
nifti_array = sitk.GetArrayFromImage(nifti_image)
# Convert the Numpy array back to a SimpleITK image
nrrd_image = sitk.GetImageFromArray(nifti_array)
# Copy information (metadata) from NIfTI image to NRRD image
nrrd_image.CopyInformation(nifti_image)
# Write the NRRD image
nrrd_writer.Execute(nrrd_image)
print("Conversion complete.")


labelpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/S-Final-1-label.nrrd"
masknrrdpath = output_nrrd_path_mask
output_path = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images/'

# 3D Screenshot Stuff check for repeats
segmentation_path = labelpath
case_number = "Case-59-"
output_path_seg = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images/'  # Change this to your desired output directory


# These filenames are just to have the program run all at the same time
rLung_filename_base = case_number + "Lungs"
Aseed_filename_base = case_number + "Aseed"

vtk_filename_rLung = f"{rLung_filename_base}.vtk"
vtk_filename_Aseed = f"{Aseed_filename_base}.vtk"
output_filename_rLung = output_path_seg + vtk_filename_rLung
output_filename_Aseed = output_path_seg + vtk_filename_Aseed
lung_screenshot_filename = "Lung-Mask-Model.png"
print("yay")
# This line creates Models Forlder if not already made
if not os.path.exists(output_path_seg):
    os.makedirs(output_path_seg)

data, header = nrrd.read(output_nrrd_path_mask)
data[data == 17] = 3
data[data == 5] = 3
print("yay")
# Load the .nrrd segmentation for Vseed
nrrd_reader_rLung = vtk.vtkNrrdReader()
nrrd_reader_rLung.SetFileName(output_nrrd_path_mask)
nrrd_reader_rLung.Update()

vtk_image_data_rLung = vtk.vtkImageData()
vtk_image_data_rLung.DeepCopy(nrrd_reader_rLung.GetOutput())

# Define scalar value for "Vseed" segment
rLung_scalar_value = 3

# Create a Marching Cubes filter for "Vseed" segment
contour_rLung = vtk.vtkMarchingCubes()
contour_rLung.SetInputData(vtk_image_data_rLung)
contour_rLung.SetValue(0, rLung_scalar_value)
contour_rLung.Update()
print("yay")
# Write the "Vseed" mesh to a .vtk file
vtk_writer_rLung = vtk.vtkPolyDataWriter()
vtk_writer_rLung.SetFileName(output_filename_rLung)
vtk_writer_rLung.SetInputData(contour_rLung.GetOutput())
vtk_writer_rLung.Write()

print("Lung conversion complete.")


#CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
screenshot_output = output_path_seg + case_number + lung_screenshot_filename
mlab.figure(bgcolor=(1, 1, 1))
srcrLung = mlab.pipeline.open(output_filename_rLung)
srcAseed = mlab.pipeline.open(output_filename_Aseed)

# Create a surface visualization of the data with blue color and 50% opacity
surfaceR = mlab.pipeline.surface(srcrLung, color=(128/255, 174/255, 128/255), opacity=.15)
surfaceAseed = mlab.pipeline.surface(srcAseed, color=(0, 151/255, 206/255), opacity=1)

# Find Segmentation Center
model = sitk.ReadImage(segmentation_path)
origin = model.GetOrigin()
dimensions = model.GetSize()
direction = model.GetDirection()
spacing = (0.38, 0.38, 0.38)
distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2

print("Origin: ", origin)
print("Dimensions: ", dimensions)
print("Center: ", center)

x, y, z = center[0], center[1], center[2]
ypt = y + 500

camera_focal_point = np.array([x, y, z])
camera_position = np.array([x,ypt,z])

print("Camera Position", camera_position)
print("Camera Focal Point", camera_focal_point)

# Set the camera parameters
# Write code to calculate this
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]

# Attach the callback to the interaction event

magnification_factor = 5
mlab.savefig(screenshot_output, magnification=magnification_factor)
 
mlab.show()

print("Process Completed")


In [ ]:
# YES WORKING TO FIND ALL THREE FILES CONTAINED WITHIN THE subdirectory 1-DONE 
import os
import pandas as pd
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import nrrd
from PIL import Image, ImageDraw
import os
import vtk
from mayavi import mlab

excel_file_path = 'z:/Q- Figures&Reports/T32-Poster/Nate/T7-Summary(LocalOld).xlsx'
# Read the Excel file
data = pd.read_excel(excel_file_path, skiprows=[0])

# Define the indices for the columns you're interested in (0-based indexing)
column2_index = 1  # Replace with the correct index for column 2
column3_index = 2  # Replace with the correct index for column 3

# Filter out rows where column 3 has a value but column 2 is blank
filtered_data = data[data.iloc[:, column2_index].notna() | data.iloc[:, column3_index].isna()]

# Extract the second and third columns from the filtered data and convert them to lists
second_column = filtered_data.iloc[:, column2_index].tolist()
third_column = filtered_data.iloc[:, column3_index].tolist()

# Remove 'NaN' values from the lists
indices = [int(value) for value in second_column if not pd.isna(value)]
case_numbers = [int(value) for value in third_column if not pd.isna(value)]

def find_files(folder_path):
    vx3_file = None
    mask_file = None
    label_file = None
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file == "Vx3.nrrd":
                vx3_file = os.path.join(root, file)
            elif file.endswith(".mask.nii.gz"):
                mask_file = os.path.join(root, file)
            elif file.endswith("label.nrrd"):
                label_file = os.path.join(root, file)
        
        if vx3_file and mask_file and label_file:
            break  # All files are found, exit the loop
    
    return vx3_file, mask_file, label_file

base_folder = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/"

for case_number in case_numbers:
    case_folder_prefix = f"Case-{case_number}-"
    
    for folder in os.listdir(base_folder):
        if folder.startswith(case_folder_prefix):
            folder_path = os.path.join(base_folder, folder)
            vx3_file, mask_file, label_file = find_files(folder_path)

            if vx3_file and mask_file and label_file:
                print("All files were found for", folder)
                print("Vx3 File:", vx3_file)
                print("Mask File:", mask_file)
                print("Label File:", label_file)
                # 200 FINAL WORKING CODE FOR IMAGE GENERATION FOR ONE CASE
                ############################################################################################################################################################
                # QC IMAGE STUFF
                # Casename = "H-14479_JH110088_SPI-JH110088-PBV_SPIROMICS_EXPIRATION_PBV__0.5__QR40__RESEARCH__A_80KV_Qr40d_3_53062428"
                Vx3Path = vx3_file
                labelpath = label_file
                maskpath = mask_file
                output_path = 'z:/Q- Figures&Reports/T32-Poster/Nate/Experiment/'
                filename = output_path + "-QC-Image.png"
                final_filename = output_path + "-QC-Image(2CM).png"
                Vx3_path = Vx3Path
                
                # 3D Screenshot Stuff check for repeats
                nrrd_file_path = labelpath
                segmentation_path = labelpath
                case_number = folder
                output_path_seg = 'z:/Q- Figures&Reports/T32-Poster/Nate/Experiment/'  # Change this to your desired output directory
                output_nrrd_file_path = "z:/Q- Figures&Reports/T32-Poster/Nate/Experiment/"

                # These filenames are just to have the program run all at the same time
                Aseed_filename_base = case_number + "Aseed"
                Vseed_filename_base = case_number + "Vseed"

                vtk_filename_aseed = f"{Aseed_filename_base}.vtk"
                vtk_filename_vseed = f"{Vseed_filename_base}.vtk"
                output_filename_aseed = output_path_seg + vtk_filename_aseed
                output_filename_vseed = output_path_seg + vtk_filename_vseed
                screenshot_filename = "Model.png"


                models_folder = os.path.join(os.path.dirname(output_path), "QC-Images")
                os.makedirs(models_folder, exist_ok=True)

                # Load the image using nibabel to see image information
                data, header = nrrd.read(Vx3_path)

                # Print the metadata
                print("Data Shape:", data.shape)
                print("Data Type:", data.dtype)
                print("Header:", header)

                # Load the 3D image
                Vx3 = sitk.ReadImage(Vx3_path)
                print("Reading Vx3...")
                # Get the size of the image in x, y, and z dimensions
                size_x, size_y, size_z = Vx3.GetSize()
                spacing = Vx3.GetSpacing()
                y_index = Vx3.GetSize()[1] // 2
                print("Spacing:", spacing)

                # Extract the middle plane along the y-dimension
                slice_2d = Vx3[:,y_index,:]
                slice_array = sitk.GetArrayFromImage(slice_2d)

                # Set the Window level
                window_level_min = -1024  # Adjust these values according to your preference
                window_level_max = 650

                # Apply Window Level to image
                windowed_image = sitk.IntensityWindowing(slice_2d, window_level_min, window_level_max, 0, 255)
                windowed_slice_array = sitk.GetArrayFromImage(windowed_image)

                #Show Vx3 with correct window level
                plt.imshow(windowed_slice_array, cmap="gray",origin='lower')
                plt.axis("off")
                plt.show()

                # Save the Corrected Vx3 .PNG to output path
                output_file_path = os.path.join(models_folder, filename)
                plt.imsave(filename, windowed_slice_array, cmap="gray", origin='lower')
                print("Image saved at:", output_file_path)

                # Load the image
                image_path = filename
                output_path_png = output_path + "/QC-Images/" + case_folder_prefix + "QC-Image(2CM).png"
                image = Image.open(image_path)

                # Determine line position
                line_start_x = 65 # 65 pixels from the left
                line_start_y = image.height - 35 # 35 pixels from the bottom

                # Calculate line end position
                line_end_x = line_start_x + int(20 / 0.38)  # 10 cm converted to pixels
                line_end_y = line_start_y

                # Draw the white line
                draw = ImageDraw.Draw(image)
                draw.line([(line_start_x, line_start_y), (line_end_x, line_end_y)], fill='white', width=2)
                # # Save the modified image
                image.save(output_path_png)

                print("Scale bar added and image saved.")

                #############################################################################################################################################################


                def update_camera_info(obj, evt):
                    camera = mlab.gcf().scene.camera
                    print("Camera Position:", camera.position)
                    print("Camera Focal Point:", camera.focal_point)
                    print("Camera View Up:", camera.view_up)


                # This line creates Models Forlder if not already made
                if not os.path.exists(output_path_seg):
                    os.makedirs(output_path_seg)


                # Load the .nrrd segmentation for Vseed
                nrrd_reader_vseed = vtk.vtkNrrdReader()
                nrrd_reader_vseed.SetFileName(segmentation_path)
                nrrd_reader_vseed.Update()

                vtk_image_data_vseed = vtk.vtkImageData()
                vtk_image_data_vseed.DeepCopy(nrrd_reader_vseed.GetOutput())

                # Define scalar value for "Vseed" segment
                vseed_scalar_value = 2

                # Create a Marching Cubes filter for "Vseed" segment
                contour_vseed = vtk.vtkMarchingCubes()
                contour_vseed.SetInputData(vtk_image_data_vseed)
                contour_vseed.SetValue(0, vseed_scalar_value)
                contour_vseed.Update()

                # Write the "Vseed" mesh to a .vtk file
                vtk_writer_vseed = vtk.vtkPolyDataWriter()
                vtk_writer_vseed.SetFileName(output_filename_vseed) 
                vtk_writer_vseed.SetInputData(contour_vseed.GetOutput())
                vtk_writer_vseed.Write()

                print("Vseed Segmentation conversion complete.")

                # Read the .nrrd file
                data, header = nrrd.read(nrrd_file_path)

                data[data == 2] = 0 
                output_nrrd_file_path = output_path + "Segmentation(Aseed).nrrd"
                # Write the modified data and header to a new .nrrd file
                nrrd.write(output_nrrd_file_path, data, header)
                print("Aseed.nrrd created!")
                # Load the .nrrd segmentation for Aseed
                nrrd_reader_aseed = vtk.vtkNrrdReader()
                nrrd_reader_aseed.SetFileName(output_nrrd_file_path)
                nrrd_reader_aseed.Update()

                vtk_image_data_aseed = vtk.vtkImageData()
                vtk_image_data_aseed.DeepCopy(nrrd_reader_aseed.GetOutput())

                # Define scalar value for "Aseed" segment
                aseed_scalar_value = 1

                # Create a Marching Cubes filter for "Aseed" segment
                contour_aseed = vtk.vtkMarchingCubes()
                contour_aseed.SetInputData(vtk_image_data_aseed)
                contour_aseed.SetValue(0, aseed_scalar_value)
                contour_aseed.Update()

                # Write the "Aseed" mesh to a .vtk file
                vtk_writer_aseed = vtk.vtkPolyDataWriter()
                vtk_writer_aseed.SetFileName(output_filename_aseed) 
                vtk_writer_aseed.SetInputData(contour_aseed.GetOutput())
                vtk_writer_aseed.Write()

                print("Aseed Segmentation Conversion Complete")

                #CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
                screenshot_output = output_path_seg + case_number + screenshot_filename

                mlab.figure(bgcolor=(1, 1, 1))
                srcA = mlab.pipeline.open(output_filename_aseed)
                srcV = mlab.pipeline.open(output_filename_vseed)

                # Create a surface visualization of the data with blue color and 50% opacity
                surfaceA = mlab.pipeline.surface(srcA, color=(0, 151/255, 206/255), opacity=1)
                surfaceV = mlab.pipeline.surface(srcV, color=(216/255, 101/255, 79/255), opacity=1)

                # Find Segmentation Center
                model = sitk.ReadImage(segmentation_path)
                origin = model.GetOrigin()
                dimensions = model.GetSize()
                direction = model.GetDirection()
                spacing = (0.38, 0.38, 0.38)
                distance = np.array(spacing) * np.array(dimensions)
                center = np.array(origin) + np.array(distance) / 2


                x, y, z = center[0], center[1], center[2]
                ypt = y + 500

                camera_focal_point = np.array([x, y, z])
                camera_position = np.array([x,ypt,z])

                # Set the camera parameters
                # Write code to calculate this
                mlab.gcf().scene.camera.position = [x, ypt, z]
                mlab.gcf().scene.camera.focal_point = [x, y, z]
                mlab.gcf().scene.camera.view_up = [0, 0, 1]

                # Attach the callback to the interaction event
                mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

                magnification_factor = 5
                mlab.savefig(screenshot_output, magnification=magnification_factor)


                print("Process Completed")

                ###################################################################################################################################################################

                # 2 to fix (nearly done)
                ## THIS CODE WORKS TO MAKE A MODEL AND POSITION IT AT A FAVORABLE ANGLE TO TAKE AN IMAGE OF THE MAYAVI REPRESENTATION OF THE MODEL

                # QC IMAGE STUFF
                # Casename = "H-14479_JH110088_SPI-JH110088-PBV_SPIROMICS_EXPIRATION_PBV__0.5__QR40__RESEARCH__A_80KV_Qr40d_3_53062428"

                # 3D Screenshot Stuff check for repeats
                nrrd_file_path = labelpath
                segmentation_path = labelpath
                # output_nrrd_file_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/Aseed.nrrd"

                # Path to the input NIfTI file
                input_nifti_path_mask = maskpath
                # Path to save the output NRRD file
                output_nrrd_path_mask = 'z:/Q- Figures&Reports/T32-Poster/Nate/Experiment/Case-59-Lung-Mask.nrrd'
                # Load the NIfTI image
                nifti_image = sitk.ReadImage(input_nifti_path_mask)
                # Create a writer for NRRD format
                nrrd_writer = sitk.ImageFileWriter()
                nrrd_writer.SetFileName(output_nrrd_path_mask)
                # Convert the NIfTI image to a Numpy array
                nifti_array = sitk.GetArrayFromImage(nifti_image)
                # Convert the Numpy array back to a SimpleITK image
                nrrd_image = sitk.GetImageFromArray(nifti_array)
                # Copy information (metadata) from NIfTI image to NRRD image
                nrrd_image.CopyInformation(nifti_image)
                # Write the NRRD image
                nrrd_writer.Execute(nrrd_image)
                print("Conversion complete.")

                masknrrdpath = output_nrrd_path_mask

                # These filenames are just to have the program run all at the same time
                rLung_filename_base = case_number + "Lungs"
                Aseed_filename_base = case_number + "Aseed"

                vtk_filename_rLung = f"{rLung_filename_base}.vtk"
                vtk_filename_Aseed = f"{Aseed_filename_base}.vtk"
                output_filename_rLung = output_path_seg + vtk_filename_rLung
                output_filename_Aseed = output_path_seg + vtk_filename_Aseed
                lung_screenshot_filename = "Lung-Mask-Model.png"
                print("yay")
                # This line creates Models Forlder if not already made
                if not os.path.exists(output_path_seg):
                    os.makedirs(output_path_seg)

                data, header = nrrd.read(output_nrrd_path_mask)
                data[data == 17] = 3
                data[data == 5] = 3
                print("yay")
                # Load the .nrrd segmentation for Vseed
                nrrd_reader_rLung = vtk.vtkNrrdReader()
                nrrd_reader_rLung.SetFileName(output_nrrd_path_mask)
                nrrd_reader_rLung.Update()

                vtk_image_data_rLung = vtk.vtkImageData()
                vtk_image_data_rLung.DeepCopy(nrrd_reader_rLung.GetOutput())

                # Define scalar value for "Vseed" segment
                rLung_scalar_value = 3

                # Create a Marching Cubes filter for "Vseed" segment
                contour_rLung = vtk.vtkMarchingCubes()
                contour_rLung.SetInputData(vtk_image_data_rLung)
                contour_rLung.SetValue(0, rLung_scalar_value)
                contour_rLung.Update()
                print("yay")
                # Write the "Vseed" mesh to a .vtk file
                vtk_writer_rLung = vtk.vtkPolyDataWriter()
                vtk_writer_rLung.SetFileName(output_filename_rLung)
                vtk_writer_rLung.SetInputData(contour_rLung.GetOutput())
                vtk_writer_rLung.Write()
                print("Lung conversion complete.")

                #CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
                screenshot_output = output_path_seg + case_number + lung_screenshot_filename
                mlab.figure(bgcolor=(1, 1, 1))
                srcrLung = mlab.pipeline.open(output_filename_rLung)
                srcAseed = mlab.pipeline.open(output_filename_Aseed)

                # Create a surface visualization of the data with blue color and 50% opacity
                surfaceR = mlab.pipeline.surface(srcrLung, color=(128/255, 174/255, 128/255), opacity=.15)
                surfaceAseed = mlab.pipeline.surface(srcAseed, color=(0, 151/255, 206/255), opacity=1)

                x, y, z = center[0], center[1], center[2]
                ypt = y + 500

                camera_focal_point = np.array([x, y, z])
                camera_position = np.array([x,ypt,z])

                print("Camera Position", camera_position)
                print("Camera Focal Point", camera_focal_point)

                # Set the camera parameters
                # Write code to calculate this
                mlab.gcf().scene.camera.position = [x, ypt, z]
                mlab.gcf().scene.camera.focal_point = [x, y, z]
                mlab.gcf().scene.camera.view_up = [0, 0, 1]

                print("Origin: ", origin)
                print("Dimensions: ", dimensions)
                print("Center: ", center)

                # Attach the callback to the interaction event

                magnification_factor = 5
                mlab.savefig(screenshot_output, magnification=magnification_factor)
                
                mlab.show()

                print("Process Completed")

            else:
                print("One or more files were not found for", folder)

            
           
            

In [13]:
from docx import Document

# Create a new Word document
doc = Document()

# Add a 2x5 table to the document
num_rows = 77
num_cols = 5
table = doc.add_table(rows=num_rows, cols=num_cols)
table.style = 'Table Grid'

# Add titles to the first row
titles = ["Index", "Case Number", "Coronal Image", "3D Segmentation", "3D Lung Mask"]
header_cells = table.rows[0].cells
for i in range(num_cols):
    header_cells[i].text = titles[i]

# Save the document
doc.save('CaseImages.docx')


In [ ]:
# ADD TO THIS. NEED TO FIX NAMING, ALSO NEED TO ADD 3D segmentations and 3D masks
from docx import Document
import glob
from PIL import Image
from docx.shared import Inches, Cm
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.table import WD_ALIGN_VERTICAL

# Create a new Word document
doc = Document()

# Add a 2x5 table to the document
num_rows = 77
num_cols = 5
table = doc.add_table(rows=num_rows, cols=num_cols)
table.style = 'Table Grid'

# Add titles to the first row
titles = ["Index", "Case Number", "Coronal Image", "3D Segmentation", "3D Lung Mask"]
header_cells = table.rows[0].cells
for i in range(num_cols):
    header_cells[i].text = titles[i]

# Folder containing the images
image_folder = 'Z:/Q- Figures&Reports/T32-Poster/Nate/Experiment/QC-Images/'

# Retrieve image files from the folder
image_files = glob.glob(image_folder + '/*.png')  # Change the extension if needed

# Percentage to resize the images (20% in this case)
resize_percentage = 0.10

# Populate the table with images, filenames, and indices
for row_index, image_path in enumerate(image_files, start=1):
    row = table.rows[row_index].cells
    img = Image.open(image_path)
    old_width = img.width
    old_height = img.height

    # Calculate new dimensions for the resized image while maintaining aspect ratio
    new_width = int(img.width * resize_percentage)
    new_height = int(img.height * resize_percentage)
    
    # Resize the image using LANCZOS resampling
    img = img.resize((new_width, new_height), Image.LANCZOS)

    filename = image_path.split('/')[-1]  # Get the filename from the full path
    
    # Populate the index column with the case number
    row[0].text = str(row_index)
    
    # Populate the filename column with the filename
    row[1].text = filename
    
    cell = row[2]  # 3rd column
    
    # Center align the cell content vertically
    cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
    
    # Add the image below the label
    cell_paragraph = cell.paragraphs[0]
    cell_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER  # Center alignment
    
    # Add the image to the paragraph
    run = cell_paragraph.add_run()
    run.add_picture(image_path, width=Cm(new_width/20))  # Use the resized image and set the width of the cell

output_path = 'Z:/Q- Figures&Reports/T32-Poster/Nate/Experiment/CaseImages.docx'
doc.save(output_path)

print(f"Document saved to: {output_path}")


In [ ]:
from docx import Document
import glob
from PIL import Image
from docx.shared import Inches, Cm
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.table import WD_ALIGN_VERTICAL

# Create a new Word document
doc = Document()

# Add a 2x5 table to the document
num_rows = 77
num_cols = 5
table = doc.add_table(rows=num_rows, cols=num_cols)
table.style = 'Table Grid'

# Add titles to the first row
titles = ["Index", "Case Number", "Coronal Image", "3D Segmentation", "3D Lung Mask"]
header_cells = table.rows[0].cells
for i in range(num_cols):
    header_cells[i].text = titles[i]

# Folder containing the images
qc_image_folder = 'Z:/Q- Figures&Reports/T32-Poster/Nate/Experiment/QC-Images/'
d3_image_folder = 'Z:/Q- Figures&Reports/T32-Poster/Nate/Experiment/3D-Images/'

# Retrieve image files from the folders
qc_image_files = glob.glob(qc_image_folder + '/*.png')
d3_image_files = glob.glob(d3_image_folder + '/*.png')

# Percentage to resize the images (20% in this case)
resize_percentage = 0.10

# Dictionary to store image paths for 3D Segmentation and 3D Lung Mask
case_images = {}

# Populate the dictionary with image paths
for image_path in d3_image_files:
    filename = image_path.split('\\')[-1]  # Use '\\' for Windows paths
    case_number = int(filename.split('-')[1])
    if "Lung-Mask" in filename:
        case_images.setdefault(case_number, {})["lung_mask"] = image_path
    elif "Model" in filename:
        case_images.setdefault(case_number, {})["model"] = image_path

# Populate the table with images, filenames, and indices
for row_index, qc_image_path in enumerate(qc_image_files, start=1):
    row = table.rows[row_index].cells
    qc_img = Image.open(qc_image_path)
    old_width = qc_img.width
    old_height = qc_img.height

    new_width = int(qc_img.width * resize_percentage)
    new_height = int(qc_img.height * resize_percentage)
    
    qc_img = qc_img.resize((new_width, new_height), Image.LANCZOS)

    filename = qc_image_path.split('\\')[-1]  # Use '\\' for Windows paths
    case_number = int(filename.split('-')[1])
    
    row[0].text = str(row_index)
    row[1].text = filename
    
    cell_qc = row[2]
    cell_qc.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
    cell_qc_paragraph = cell_qc.paragraphs[0]
    cell_qc_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    run_qc = cell_qc_paragraph.add_run()
    run_qc.add_picture(qc_image_path, width=Cm(new_width/20))
    
    if case_number in case_images:
        cell_seg = row[3]
        cell_seg.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        cell_seg_paragraph = cell_seg.paragraphs[0]
        cell_seg_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        run_seg = cell_seg_paragraph.add_run()
        if "lung_mask" in case_images[case_number]:
            seg_image_path = case_images[case_number]["lung_mask"]
            run_seg.add_picture(seg_image_path, width=Cm(new_width/20))

        cell_mask = row[4]
        cell_mask.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        cell_mask_paragraph = cell_mask.paragraphs[0]
        cell_mask_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        run_mask = cell_mask_paragraph.add_run()
        if "model" in case_images[case_number]:
            mask_image_path = case_images[case_number]["model"]
            run_mask.add_picture(mask_image_path, width=Cm(new_width/20))

output_path = 'Z:/Q- Figures&Reports/T32-Poster/Nate/Experiment/CaseImages.docx'
doc.save(output_path)

print(f"Document saved to: {output_path}")


In [ ]:
import os

# List of case numbers
 # Add more case numbers if needed

# Root directory
root_directory = "Z:/Q- Figures&Reports/T32-Poster/Nate/Experiment/"

# Search for the image file in each case folder
for case_number in case_numbers:
    case_folder = os.path.join(root_directory, f"Case-{case_number}")
    
    # Check if the case folder exists
    if os.path.exists(case_folder) and os.path.isdir(case_folder):
        image_found = False
        
        # Search for the image file within the case folder
        for file_name in os.listdir(case_folder):
            if file_name.endswith("Lung-Mask.png"):
                image_found = True
                image_file_path = os.path.join(case_folder, file_name)
                print(f"Found image file in Case-{case_number}: {image_file_path}")
                break  # Stop searching once the image file is found
        
        if not image_found:
            print(f"Image file not found in Case-{case_number}")


In [ ]:
import os

# Root directory
root_directory = "Z:/Q- Figures&Reports/T32-Poster/Nate/Experiment/"

# Recursive function to rename files
def rename_files(directory):
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            if filename.endswith("Lung-Mask-Model.png"):
                old_path = os.path.join(dirpath, filename)
                new_filename = filename.replace("Lung-Mask-Model.png", "Lung-Mask.png")
                new_path = os.path.join(dirpath, new_filename)
                os.rename(old_path, new_path)
                print(f"Renamed {filename} to {new_filename}")

# Call the function to rename files in the specified directory and its subdirectories
rename_files(root_directory)


In [ ]:
import os

# List of case numbers
  # Add more case numbers if needed

# Root directory
root_directory = "Z:/Q- Figures&Reports/T32-Poster/Nate/Experiment/"

# Search for the image files in each case folder
for case_number in case_numbers:
    case_folder = os.path.join(root_directory, f"Case-{case_number}")
    
    # Check if the case folder exists
    if os.path.exists(case_folder) and os.path.isdir(case_folder):
        lung_mask_found = False
        model_found = False
        
        # Search for the image files within the case folder
        for file_name in os.listdir(case_folder):
            if file_name.endswith("Lung-Mask.png"):
                lung_mask_found = True
                lung_mask_file_path = os.path.join(case_folder, file_name)
            
            if file_name.endswith("Model.png"):
                model_found = True
                model_file_path = os.path.join(case_folder, file_name)
            
            # If both files are found, no need to continue searching
            if lung_mask_found and model_found:
                
                break
        
        if lung_mask_found and model_found:
            print(f"Both Lung-Mask.png and Model.png files found in Case-{case_number}")
        elif lung_mask_found:
            print(f"Only Lung-Mask.png found in Case-{case_number}")
        elif model_found:
            print(f"Only Model.png found in Case-{case_number}")
        else:
            print(f"Neither Lung-Mask.png nor Model.png found in Case-{case_number}")


In [ ]:
import os
import shutil

# List of case numbers
 # Add more case numbers if needed

# Root directory
root_directory = "Z:/Q- Figures&Reports/T32-Poster/Nate/Experiment/"

# Directory for 3D Images
output_directory = os.path.join(root_directory, "3D Images")
os.makedirs(output_directory, exist_ok=True)

# Search for the image files in each case folder
for case_number in case_numbers:
    case_folder = os.path.join(root_directory, f"Case-{case_number}")
    
    # Check if the case folder exists
    if os.path.exists(case_folder) and os.path.isdir(case_folder):
        lung_mask_found = False
        model_found = False
        
        # Search for the image files within the case folder
        for file_name in os.listdir(case_folder):
            if file_name.endswith("Lung-Mask.png"):
                lung_mask_found = True
                lung_mask_file_path = os.path.join(case_folder, file_name)
            
            if file_name.endswith("Model.png"):
                model_found = True
                model_file_path = os.path.join(case_folder, file_name)
            
            # If both files are found, no need to continue searching
            if lung_mask_found and model_found:
                break
        
        if lung_mask_found and model_found:
            print(f"Both Lung-Mask.png and Model.png files found in Case-{case_number}")
            
            # Copy files to the output directory
            shutil.copy2(lung_mask_file_path, output_directory)
            shutil.copy2(model_file_path, output_directory)
            
        elif lung_mask_found:
            print(f"Only Lung-Mask.png found in Case-{case_number}")
            shutil.copy2(lung_mask_file_path, output_directory)
            
        elif model_found:
            print(f"Only Model.png found in Case-{case_number}")
            shutil.copy2(model_file_path, output_directory)
            
        else:
            print(f"Neither Lung-Mask.png nor Model.png found in Case-{case_number}")


In [112]:
from docx import Document
import glob
from PIL import Image
from docx.shared import Inches, Cm
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.table import WD_ALIGN_VERTICAL

doc = Document()
num_rows = 77
num_cols = 5
table = doc.add_table(rows=num_rows, cols=num_cols)
table.style = 'Table Grid'

titles = ["Index", "Case Number", "Coronal Image", "3D Segmentation", "3D Lung Mask"]
header_cells = table.rows[0].cells
for i in range(num_cols):
    header_cells[i].text = titles[i]

image_folder = 'Z:/Q- Figures&Reports/T32-Poster/Nate/Experiment/QC-Images/'
lung_mask_folder = 'Z:/Q- Figures&Reports/T32-Poster/Nate/Experiment/3D Images/'
model_image_folder = 'Z:/Q- Figures&Reports/T32-Poster/Nate/Experiment/3d Images/'

image_files = glob.glob(image_folder + '/*.png')
lung_mask_files = glob.glob(lung_mask_folder + '/*Lung-Mask.png')
model_image_files = glob.glob(model_image_folder + '/*Model.png')

# resize_percentage = 0.10

for row_index, (image_path, lung_mask_path, model_image_path) in enumerate(zip(image_files, lung_mask_files, model_image_files), start=1):
    row = table.rows[row_index].cells
    img = Image.open(image_path)
    lung_mask_img = Image.open(lung_mask_path)
    model_img = Image.open(model_image_path)
    
    # old_width = img.width
    # old_height = img.height

    new_width = int(img.width)
    # new_height = int(img.height * resize_percentage)
    
    # img = img.resize((new_width, new_height), Image.LANCZOS)
    
    filename = image_path.split('/')[-1]

    row[0].text = str(row_index)
    row[1].text = filename
    
    cell = row[2]
    cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
    cell_paragraph = cell.paragraphs[0]
    cell_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    run = cell_paragraph.add_run()
    run.add_picture(image_path, width=Cm(new_width/300))
    
    model_cell = row[3]  # 4th column
    model_cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
    model_paragraph = model_cell.paragraphs[0]
    model_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    model_run = model_paragraph.add_run()
    model_run.add_picture(model_image_path, width=Cm(new_width/300))
    
    lung_mask_cell = row[4]
    lung_mask_cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
    lung_mask_paragraph = lung_mask_cell.paragraphs[0]
    lung_mask_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    lung_mask_run = lung_mask_paragraph.add_run()
    lung_mask_run.add_picture(lung_mask_path, width=Cm(new_width/300))

output_path = 'Z:/Q- Figures&Reports/T32-Poster/Nate/Experiment/CaseImages.docx'
doc.save(output_path)
print(f"Document saved to: {output_path}")


Document saved to: Z:/Q- Figures&Reports/T32-Poster/Nate/Experiment/CaseImages.docx
